# Using a Multilayer 

We will now apply our knowledge of different layers to real data with using a multilayer neural network on the Low Birthweight dataset.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.framework import ops

In [2]:
BATCH_SIZE = 100

In [3]:
def create_weight(shape, std=1, name='weight'):
    return tf.Variable(tf.random_normal(shape=shape, mean=0, stddev=std), name=name)

def create_bias(shape, std=1, name='bias'):
    return tf.Variable(tf.random_normal(shape=shape, mean=0, stddev=std), name=name)

def fully_connected(inputs, weights, biases, name='fully_connected'):
    full = tf.add(tf.matmul(inputs, weights), biases)
    return tf.nn.relu(full, name=name)

### Downloading data

In [6]:
DATA_URL = 'https://www.umass.edu/statdata/statdata/data/lowbwt.dat'
DATA_FILE = 'remote_data/birthdata.txt'

In [ ]:
%mkdir remote_data

In [ ]:
!wget {DATA_URL} -O {DATA_FILE}

### Loading data

In [7]:
widths = [4, 7, 7, 7, 8, 9, 7, 6, 6, 7, 8]
COLS_OF_INTEREST = ['AGE', 'LWT', 'RACE', 'SMOKE', 'PTL', 'HT', 'UI', 'FTV', 'BWT']

df = pd.read_fwf(DATA_FILE, widths=widths, skiprows=5, skipfooter=1, skip_blank_lines=True, usecols=COLS_OF_INTEREST)
df = df.drop(0)

In [8]:
print(df.shape)
df.head(3)

(189, 9)


,AGE,LWT,RACE,SMOKE,PTL,HT,UI,FTV,BWT
1,19.0,182.0,2.0,0.0,0.0,0.0,1.0,0.0,2523.0
2,33.0,155.0,3.0,0.0,0.0,0.0,0.0,3.0,2551.0
3,20.0,105.0,1.0,1.0,0.0,0.0,0.0,1.0,2557.0


### Normalizing data

In [9]:
df.fillna(df.mean(), inplace=True)
df.head(3)

,AGE,LWT,RACE,SMOKE,PTL,HT,UI,FTV,BWT
1,19.0,182.0,2.0,0.0,0.0,0.0,1.0,0.0,2523.0
2,33.0,155.0,3.0,0.0,0.0,0.0,0.0,3.0,2551.0
3,20.0,105.0,1.0,1.0,0.0,0.0,0.0,1.0,2557.0


### Splitting training and test sets

In [10]:
seed = 4
tf.set_random_seed(seed)
np.random.seed(seed)

In [11]:
N_SAMPLES = df.shape[0]
N_TRAIN = int(N_SAMPLES * 0.8)

print(N_SAMPLES)
print(N_TRAIN)

189
151


In [12]:
idx = np.arange(N_SAMPLES)
np.random.shuffle(idx)

idx_train = idx[: N_TRAIN]
idx_test = idx[N_TRAIN :]

df_train = df.loc[idx_train]
df_test = df.loc[idx_test]

In [13]:
print(df_train.shape)
df_train.head(3)

(151, 9)


,AGE,LWT,RACE,SMOKE,PTL,HT,UI,FTV,BWT
33,35.0,121.0,2.0,1.0,1.0,0.0,0.0,1.0,2948.0
82,32.0,170.0,1.0,0.0,0.0,0.0,0.0,0.0,3473.0
166,20.0,121.0,1.0,1.0,1.0,0.0,1.0,0.0,2296.0


In [14]:
print(df_test.shape)
df_test.head(3)

(38, 9)


,AGE,LWT,RACE,SMOKE,PTL,HT,UI,FTV,BWT
95,16.0,135.0,1.0,1.0,0.0,0.0,0.0,0.0,3643.0
98,19.0,147.0,1.0,1.0,0.0,0.0,0.0,0.0,3651.0
115,20.0,170.0,1.0,1.0,0.0,0.0,0.0,0.0,3940.0


### Splitting inputs and labels

In [15]:
COLS_INPUT = ['AGE', 'LWT', 'RACE', 'SMOKE', 'PTL', 'HT', 'UI', 'FTV']
COL_LABEL = ['BWT']

inputs_train = df_train[COLS_INPUT]
labels_train = df_train[COL_LABEL]
inputs_test = df_test[COLS_INPUT]
labels_test = df_test[COL_LABEL]

print(inputs_train.shape)
print(labels_train.shape)
print(inputs_test.shape)
print(labels_test.shape)

(151, 8)
(151, 1)
(38, 8)
(38, 1)


### Resetting graph and creating session

In [54]:
ops.reset_default_graph()
sess = tf.Session()

### Initializing input placeholders

In [55]:
X = tf.placeholder(shape=[None, 8], dtype=tf.float32, name='X')
y = tf.placeholder(shape=[None, 1], dtype=tf.float32, name='y')

print(X)
print(y)

Tensor("X:0", shape=(?, 8), dtype=float32)
Tensor("y:0", shape=(?, 1), dtype=float32)


### Layer 1 (25 hidden nodes)

In [56]:
with tf.name_scope('layer_1'):
    w1 = create_weight(shape=[8, 25], std=10.0, name='w1')
    b1 = create_bias(shape=[25], std=10.0, name='b1')
    layer1 = fully_connected(inputs=X, weights=w1, biases=b1, name='layer1')
    
print(w1)
print(b1)
print(layer1)

Tensor("layer_1/w1/read:0", shape=(8, 25), dtype=float32)
Tensor("layer_1/b1/read:0", shape=(25,), dtype=float32)
Tensor("layer_1/layer1:0", shape=(?, 25), dtype=float32)


### Layer 2 (10 hidden nodes)

In [57]:
with tf.name_scope('layer_2'):
    w2 = create_weight(shape=[25, 10], std=10.0, name='w2')
    b2 = create_bias(shape=[10], std=10.0, name='b2')
    layer2 = fully_connected(inputs=layer1, weights=w2, biases=b2, name='layer2')
    
print(w2)
print(b2)
print(layer2)

Tensor("layer_2/w2/read:0", shape=(25, 10), dtype=float32)
Tensor("layer_2/b2/read:0", shape=(10,), dtype=float32)
Tensor("layer_2/layer2:0", shape=(?, 10), dtype=float32)


### Layer 3 (3 hidden nodes)

In [58]:
with tf.name_scope('layer_3'):
    w3 = create_weight(shape=[10, 3], std=10.0, name='w3')
    b3 = create_bias(shape=[3], std=10.0, name='b3')
    layer3 = fully_connected(inputs=layer2, weights=w3, biases=b3, name='layer3')
    
print(w3)
print(b3)
print(layer3)

Tensor("layer_3/w3/read:0", shape=(10, 3), dtype=float32)
Tensor("layer_3/b3/read:0", shape=(3,), dtype=float32)
Tensor("layer_3/layer3:0", shape=(?, 3), dtype=float32)


### Output layer

In [59]:
with tf.name_scope('output'):
    w4 = create_weight(shape=[3, 1], std=10.0, name='w4')
    b4 = create_bias(shape=[1], std=10.0, name='b4')
    output = fully_connected(inputs=layer3, weights=w4, biases=b4, name='output')
#     loss = tf.reduce_mean(tf.abs(output - y), name='loss')
    loss = tf.reduce_mean(tf.square(output - y), name='loss')
    
print(w4)
print(b4)
print(output)
print(loss)

Tensor("output/w4/read:0", shape=(3, 1), dtype=float32)
Tensor("output/b4/read:0", shape=(1,), dtype=float32)
Tensor("output/output:0", shape=(?, 1), dtype=float32)
Tensor("output/loss:0", shape=(), dtype=float32)


### Loss and training step

In [60]:
# optimizer = tf.train.AdamOptimizer(0.05)
optimizer = tf.train.GradientDescentOptimizer(0.05)
train = optimizer.minimize(loss)

### Initializing variables

In [61]:
init = tf.global_variables_initializer()
sess.run(init)

### Training

In [62]:
losses_train = []
losses_test = []


ITERATIONS = 20000
LOG_STEP = 2000
feed_dict_test = {X: inputs_test, y: labels_test}

for i in range(ITERATIONS):
    idx_rand = np.random.choice(idx_train, size=BATCH_SIZE, replace=False)
#     print(idx_rand[: 6])
#     idx_rand = idx_train[: 6]
    inputs_rand = inputs_train.loc[idx_rand]
    labels_rand = labels_train.loc[idx_rand]
    feed_dict_train = {X: inputs_rand, y: labels_rand}
    sess.run(train, feed_dict=feed_dict_train)
    
#     l1, l2, l3, ou = sess.run([layer1, layer2, layer3, output], feed_dict=feed_dict_train)
#     print('Layer1:', l1)
#     print('Layer2:', l2)
#     print('Layer3:', l3)
#     print('Output:', ou)
    
    loss_train = sess.run(loss, feed_dict=feed_dict_train)
    losses_train.append(loss_train)
    
    loss_test = sess.run(loss, feed_dict=feed_dict_test)
    losses_test.append(loss_test)
    
    if i == 0 or (i + 1) % LOG_STEP == 0:
        print('#{} - loss: {}'.format(i + 1, loss_train))

#1 - loss: 8988466.0
#2000 - loss: 9231564.0
#4000 - loss: 9319165.0
#6000 - loss: 8945099.0
#8000 - loss: 8926106.0
#10000 - loss: 8770189.0
#12000 - loss: 8811158.0
#14000 - loss: 9168930.0
#16000 - loss: 8854388.0
#18000 - loss: 8837446.0
#20000 - loss: 9189558.0


In [ ]:
sess.run(w3)

In [ ]:
sess.run(b3)

In [ ]:
sess.run(tf.matmul(layer2, w3) + b3, feed_dict={X: inputs_rand, y:labels_rand})

In [ ]:
aa = pd.DataFrame([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], columns=['Value'])
aa

In [ ]:
idx = np.arange(aa.shape[0])
idx

In [ ]:
np.random.shuffle(idx)
idx

In [ ]:
idx1 = idx[: 8]
idx1

In [ ]:
idx2 = idx[8 :]
idx2

In [ ]:
aat = aa.loc[idx1]
aat

In [ ]:
aae = aa.loc[idx2]
aae

In [ ]:
idx_rand = np.random.choice(idx1, size=4, replace=False)
idx_rand

In [ ]:
aar = aat.loc[idx_rand]
aar